In [1]:
import mlflow
import os
from urllib.parse import urlparse

In [2]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [5]:
# Set the MLflow tracking URI to your DagsHub repository
mlflow.set_tracking_uri("https://dagshub.com/h.sahni1998/end-to-end-ML-with-MLflow.mlflow")

# Set the authentication token (ensure no trailing spaces)
os.environ["MLFLOW_TRACKING_USERNAME"] = "h.sahni1998"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0dbf3973dc39db1bcd38323686b0d68db93c4655"

In [6]:
print("MLFLOW_TRACKING_USERNAME:", os.environ.get("MLFLOW_TRACKING_USERNAME"))
print("MLFLOW_TRACKING_PASSWORD:", os.environ.get("MLFLOW_TRACKING_PASSWORD"))
print("MLFLOW_TRACKING_URI:", mlflow.get_tracking_uri())


MLFLOW_TRACKING_USERNAME: h.sahni1998
MLFLOW_TRACKING_PASSWORD: 0dbf3973dc39db1bcd38323686b0d68db93c4655
MLFLOW_TRACKING_URI: https://dagshub.com/h.sahni1998/end-to-end-ML-with-MLflow.mlflow


In [7]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class ModelEvaluationConfig:
    """
    Configuration class for model evaluation.

    Attributes:
        root_dir (Path): The root directory where model evaluation operations will take place.
        test_data_path (Path): The path to the testing data.
        model_path (Path): The path to the saved model that will be evaluated.
        all_params (dict): A dictionary containing parameters for the evaluation.
        metric_file_name (Path): The path to the file where evaluation metrics will be saved.
        target_column (str): The name of the target column in the dataset.
        mlflow_uri (str): The URI for MLflow tracking.
    """
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Get the ModelEvaluationConfig object from the configuration.
        
        Returns:
            ModelEvaluationConfig: Configuration for model evaluation.
        """
        config = self.config.model_evaluation  # Get the model evaluation configuration section
        params = self.params.ElasticNet  # Get the ElasticNet parameters
        schema = self.schema.TARGET_COLUMN  # Get the target column from the schema

        create_directories([config.root_dir])  # Ensure the root directory for model evaluation exists

        # Create and return a ModelEvaluationConfig object with the configuration details
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/h.sahni1998/end-to-end-ML-with-MLflow.mlflow",
        )

        return model_evaluation_config


In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
import pandas as pd  # Import pandas for data manipulation
import numpy as np  # Import numpy for numerical operations
import joblib  # Import joblib for loading the model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # Import metrics for evaluation
import mlflow  # Import mlflow for tracking and logging
from urllib.parse import urlparse  # Import urlparse for parsing URIs
from pathlib import Path  # Import Path class for handling filesystem paths

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        """
        Initialize the ModelEvaluation class with a configuration.
        
        Args:
            config (ModelEvaluationConfig): Configuration for model evaluation.
        """
        self.config = config  # Store the configuration

    def eval_metrics(self, actual, pred):
        """
        Evaluate the model performance using RMSE, MAE, and R2 score.
        
        Args:
            actual (array-like): Actual target values.
            pred (array-like): Predicted target values.
        
        Returns:
            tuple: RMSE, MAE, and R2 score.
        """
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        """
        Log the model evaluation metrics and model into MLflow.
        """
        # Load the test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split the test data into features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set the MLflow tracking URI
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Start an MLflow run
        with mlflow.start_run():
            # Predict the target values using the loaded model
            predicted_qualities = model.predict(test_x)

            # Evaluate the model performance
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Save metrics locally as JSON
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Log the model to MLflow
            if tracking_url_type_store != "file":
                # Register the model if not using file store
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [12]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Retrieve model evaluation configuration
    model_evaluation_config = config.get_model_evaluation_config()
    
    # Initialize ModelEvaluation with the retrieved configuration
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    
    # Log metrics and model into MLflow
    model_evaluation.log_into_mlflow()
except Exception as e:
    # Raise the exception if any error occurs
    raise e


[2024-06-13 16:39:09,054: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-13 16:39:09,055: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-13 16:39:09,058: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-13 16:39:09,058: INFO: common: Created directory at: artifacts]
[2024-06-13 16:39:09,059: INFO: common: Created directory at: artifacts/model_evaluation]
[2024-06-13 16:39:09,602: INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]


/home/himanshu/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/06/13 16:39:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
